## Установка библиотек

In [ ]:
# установка библиотек
!pip install -q pyshorteners
!pip install -qU langchain-text-splitters
!pip install -q faiss-cpu langchain openai tiktoken langchain_community langchain-openai
!pip install -q httpx==0.27.2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
# импортируем библиотеки
from bs4 import BeautifulSoup
import requests
import pyshorteners
import json
import re
from pprint import pprint
from langchain_text_splitters import HTMLHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from openai import OpenAI

## Парсинг сайта Google Analytics
НЕ ЗАПУСКАТЬ ЭТОТ РАЗДЕЛ! Векторная база уже создана.

In [ ]:
# список ссылок для парсинга
list_to_parse = ['https://support.google.com/analytics/', 'https://support.google.com/analytics/community/', 'https://support.google.com/analytics/announcements/10707884']

In [ ]:
#извлекаем названия разделов
def extract_section_name(url):
  match = re.search(r'analytics/([^/]+)/?', url)
  if match:
    return match.group(1)
  else:
    return 'analytics'

In [ ]:
#парсим страницы

#создаем множество из уникальных ссылок
set_parsed = set(list_to_parse)
ind = 0

for url in list_to_parse:
  section_name = extract_section_name(url)
  response = requests.get(url)
  bs = BeautifulSoup(response.text, "html.parser")

  #ищем ссылки на страницах
  #main_cont = bs.findAll('div', class_='main-content')
  links = bs.findAll('a',href=True)

  #добавляем в наше множество новые ссылки если их там нет
  for link in links:
    href = link.get('href')
    if (href not in set_parsed) and (href not in list_to_parse) and section_name in href:
      if not href.startswith('/analytics'):
        continue
      href = 'https://support.google.com' + href

      href_un = href.split('?')[0].split('#')[0]
      if href_un not in set_parsed:
          set_parsed.add(href_un)
          list_to_parse.append(href_un)
          print(ind, href_un)
          ind += 1
  if ind > 10000: break

0 https://support.google.com/analytics
1 https://support.google.com/analytics/community
2 https://support.google.com/analytics/gethelp
3 https://support.google.com/analytics/topic/14089939
4 https://support.google.com/analytics/topic/14088998
5 https://support.google.com/analytics/answer/1009692
6 https://support.google.com/analytics/topic/14089317
7 https://support.google.com/analytics/topic/13614110
8 https://support.google.com/analytics/answer/6367342
9 https://support.google.com/analytics/answer/14906673
10 https://support.google.com/analytics/topic/9355633
11 https://support.google.com/analytics/topic/14710097
12 https://support.google.com/analytics/topic/10054560
13 https://support.google.com/analytics/topic/9306488
14 https://support.google.com/analytics/topic/12270146
15 https://support.google.com/analytics/topic/12943241
16 https://support.google.com/analytics/topic/14272008
17 https://support.google.com/analytics/answer/9900444
18 https://support.google.com/analytics/answer/1

In [ ]:
print(len(list_to_parse))
# Сохранение списка ссылок в JSON файл
with open('data.json', 'w') as file:
    json.dump(list_to_parse, file, indent=4) # indent=4 для красивого форматирования

1155


In [ ]:
from google.colab import drive

# Это примонтирует ваш Google Drive к Colab
drive.mount('/content/drive')

# Путь к папке, где сохранены файлы
folder_path  = "/content/drive/My Drive/"


Mounted at /content/drive


In [ ]:
# Чтение списка ссылок из JSON файла
with open('/content/drive/My Drive/data.json', 'r') as file:
    data = json.load(file)

print(data)


['https://support.google.com/analytics/', 'https://support.google.com/analytics/community/', 'https://support.google.com/analytics/announcements/10707884', 'https://support.google.com/analytics', 'https://support.google.com/analytics/community', 'https://support.google.com/analytics/gethelp', 'https://support.google.com/analytics/topic/14089939', 'https://support.google.com/analytics/topic/14088998', 'https://support.google.com/analytics/answer/1009692', 'https://support.google.com/analytics/topic/14089317', 'https://support.google.com/analytics/topic/13614110', 'https://support.google.com/analytics/answer/6367342', 'https://support.google.com/analytics/answer/14906673', 'https://support.google.com/analytics/topic/9355633', 'https://support.google.com/analytics/topic/14710097', 'https://support.google.com/analytics/topic/10054560', 'https://support.google.com/analytics/topic/9306488', 'https://support.google.com/analytics/topic/12270146', 'https://support.google.com/analytics/topic/129

In [ ]:
data[100]

'https://support.google.com/analytics/answer/12159453'

In [ ]:
# посмотрим на одну из статей в html разметке
response = requests.get(data[100])
soup = BeautifulSoup(response.text, "html.parser")
article = soup.find('article')
article

<article class="article page" itemscope="" sc-render-smart-button="false"> <section class="article-container" data-stats-ve="35"><h1>The value of digital analytics</h1><div class="article-content-container"><div class="cc"><p>In marketing, customer engagement is measured using something called a <strong>purchase funnel</strong>. It shows the different stages that customers experience with your business, such as:</p>
<ul>
<li><strong>Acquisition</strong>: The initial stage of building business awareness and acquiring user interest</li>
<li><strong>Behavior</strong>: The interactions of users engaging with your business</li>
<li><strong>Conversion</strong>: When a user becomes a customer and transacts with your business</li>
</ul>
<p><img alt="" data-alt-src="//lh3.googleusercontent.com/dWu1SKVLPYKtn4F8yC-6rzZmt6MXi-Gz5ctQimLYh6dy2euozQe2duyiCgEg56_RW89a" data-mime-type="image/png" height="280" src="//lh3.googleusercontent.com/dWu1SKVLPYKtn4F8yC-6rzZmt6MXi-Gz5ctQimLYh6dy2euozQe2duyiCgEg5

## Разбиваем на чанки напрямую из ссылок
НЕ ЗАПУСКАТЬ ЭТОТ РАЗДЕЛ! Векторная база уже создана.

In [ ]:
# Используем HTMLHeaderTextSplitter, чтобы разбить по html заголовкам
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on)

In [ ]:
# Проходимся по каждой ссылке и разбиваем текст в HTML-разметке на чанки
results = []

for url in data:
  try:
    text = html_splitter.split_text_from_url(url)
    results.append((text))
  except Exception as e:
    print(f"Error processing {url}: {e}")

In [ ]:
results[10]

[Document(metadata={}, page_content='Analytics Help  \nSign in  \nGoogle Help  \nHelp CenterCommunityAnalytics  \nGet started with AnalyticsCollect and manage dataReport and exploreAdvertising and attributionAudiences and remarketingManage accounts, properties, and usersGoogle Analytics 360Migrate from UA to GA4 [Legacy]Policies and data privacy  \nPrivacy PolicyTerms of ServiceSubmit feedback  \nSend feedback on...  \nThis help content & information  \nGeneral Help Center experience  \nNext  \nHelp CenterCommunityAnnouncements  \nAnalytics Contact us  \nGet started with AnalyticsCollect and manage dataReport and exploreAdvertising and attributionAudiences and remarketingManage accounts, properties, and usersGoogle Analytics 360Migrate from UA to GA4 [Legacy]Policies and data privacy  \nIntroduction to AnalyticsSet up AnalyticsGuides and videosTroubleshootGlossary  \nEvents and key eventsImport data from external systemsIntegrationsMeasure ecommerceMeasure lead generation[GA4] First-pa

In [ ]:
# дополнительно разбиваем на более мелкие чанки
chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

flattened_results = [doc for sublist in results for doc in sublist]

# Splits
splits = text_splitter.transform_documents(flattened_results)

In [ ]:
len(splits)

29113

In [ ]:
splits[10000]

Document(metadata={'Header 1': '[GA4] Create user-scoped custom dimensions', 'Header 2': 'Instructions'}, page_content='In Admin, under Data display, click Custom definitions. Select the Custom dimensions tab Click Create custom dimensions. Complete the following fields: Click Save.  \nNote: The previous link opens to the last Analytics property you accessed. You can change the property using the property selector. You must be an Editor or above at the property level to\xa0create user-scoped custom dimensions.')

## Ключ API OpenAI

In [ ]:
import getpass
import os
import openai
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


## Создание векторной базы
НЕ ЗАПУСКАТЬ ЭТОТ РАЗДЕЛ! ВЕКТОРНАЯ БАЗА УЖЕ СОЗДАНА.

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()
# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(splits, embeddings)

In [ ]:
# сохранение вектрной базы на драйв
from google.colab import drive

# Это примонтирует ваш Google Drive к Colab
drive.mount('/content/drive')

# сохранить файл напрямую в Google Drive
folder_path  = "/content/drive/My Drive/"

# Задаем имя, чтобы идентифицировать сохраненные файлы
index_name = "db_from_docs_PP"

# сохраняем db_from_texts на ваш гугл драйв
db.save_local(folder_path=folder_path, index_name=index_name)

## Загружаем сохраненную векторную базу

In [ ]:
# подключаемся к google drive
from google.colab import drive

# Это примонтирует ваш Google Drive к Colab
drive.mount('/content/drive')

# Путь к папке, где сохранены файлы индекса и хранилища документов, нужно закинуть предварительно два файла векторной базы в эту папку
folder_path  = "/content/drive/My Drive/"

# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Имя, используемое при сохранении файлов
index_name = "db_from_docs_PP"

# Загрузка данных и создание нового экземпляра векторной базы faiss_instance
faiss_instance = FAISS.load_local(
    folder_path=folder_path,
    embeddings=embeddings,
    index_name=index_name,
    allow_dangerous_deserialization=True
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ФИНАЛЬНАЯ Модель

In [ ]:
# системный промпт на русском
system = '''Ты - нейро-эксперт по Google Analytics (GA4), платформе для сбора данных с сайтов и приложений и создания отчетов с информацией, полезной для разных бизнесов.
Тебе будет представлена следующая информация: саммаризированный предыдущий диалог с пользователем, актуальный вопрос
пользователя, документы с информацией для ответа клиенту. Твоя задача - полно и точно ответить на вопрос пользователя по
поводу платформы Google Analytics, ее функциях, возможностях, установке.  Особенно аккуратно отвечай по цифрам - лимитам и тому, что входит
в каждую версию подписки. В своем ответе основывайся только на представленных тебе документах, не придумывай ничего “от себя”, однако
пользователю об этих документах и их номерах не сообщай. Если пользователь представился (назвал свое имя), используй его имя при ответе.
Если в кратком обзоре предыдущего диалога фигурирует имя пользователя, обязательно используй его в ответе, обращайся к пользователю по имени.
Тебе запрещено здороваться с пользователем'''


In [ ]:
# Функция ответа модели
def answer_index(system, search_query, topic, search_index, verbose=0):
    # Поиск релевантных отрезков из базы знаний по вопросу пользователя
    docs = search_index.similarity_search(search_query, k=8)
    if verbose:
        print('\n ===========================================: ')

    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose:
        print('message_content :\n ======================================== \n', message_content)

    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос пользователя. Документы с информацией для ответа клиенту: {message_content}\n\n{topic}"}
    ]

    if verbose:
        print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0
    )
    answer = completion.choices[0].message.content
    return answer

In [ ]:
# Функция саммаризации предыдущего диалога
def summarize_questions(dialog):
    """
    Функция возвращает саммаризированный текст диалога.
    """
    client = OpenAI()
    messages = [
        {"role": "system", "content": "Ты - нейро-саммаризатор. Твоя задача - саммаризировать диалог, который тебе пришел. Если пользователь назвал свое имя, обязательно отрази его в саммаризированном диалоге"},
        {"role": "user", "content": "Саммаризируй следующий диалог консультанта и пользователя, тебе запрещено удалять из саммаризации имя пользователя: " + " ".join(dialog)}
    ]

    completion = client.chat.completions.create(
        model="gpt-4o-mini",     # используем gpt4 для более точной саммаризации
        messages=messages,
        temperature=0,          # Используем более низкую температуру для более определенной суммаризации
    )

    return completion.choices[0].message.content

In [ ]:
# Основная функция ответа модели с учетом саммаризированного предыдущего диалога
def answer_user_question_dialog(system, db, user_question, question_history):
    """
    Функция возвращает ответ на вопрос пользователя.
    """
    summarized_history = ""
    # Если в истории более одного вопроса, применяем суммаризацию
    if len(question_history) > 0:
        summarized_history = "Вот саммаризированный предыдущий диалог с пользователем: " +summarize_questions([q + ' ' + (a if a else '') for q, a in question_history])

    topic = summarized_history + " Актуальный вопрос пользователя: " + user_question

    # Получаем ответ, используя только user_question для поиска в базе данных
    answer_text = answer_index(system, user_question, topic, db)

    question_history.append((user_question, answer_text if answer_text else ''))
    # Выводим саммаризированный текст, который видит модель
    if summarized_history:
        print('****************************')
        print(summarized_history)
        print('****************************')

    return answer_text

In [ ]:
# форматируем текст ответа
def insert_newlines(text: str, max_len: int = 170) -> str:
    """
    Функция разбивает длинный текст на строки определенной максимальной длины.
    """
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
# Функция для создания диалога с пользователем
def run_dialog(system_doc_url, knowledge_base_url):
    """
    Функция запускает диалог между пользователем и нейро-консультантом.
    """
    #список кортежей, где каждый кортеж содержит пару вопрос-ответ, для отслеживания истории вопросов и ответов во время сессии диалога.
    question_history = []
    while True:
        user_question = input('Пользователь: ')
        if user_question.lower() == 'stop':
            break
        answer = answer_user_question_dialog(system_doc_url, knowledge_base_url, user_question, question_history)
        print('Консультант:', insert_newlines(answer))

    return

In [ ]:
# запускаем диалог
run_dialog(system, faiss_instance)

Пользователь: привет
Консультант:  Не могу ответить на ваш вопрос.
Пользователь: Привет меня зовут Дана
****************************
Вот саммаризированный предыдущий диалог с пользователем: Пользователь приветствует консультанта, но консультант не может ответить на его вопрос.
****************************
Консультант:  Дана, как я могу помочь вам с вопросами по Google Analytics?
Пользователь: What is Google Analytics and why would businesses need it?
****************************
Вот саммаризированный предыдущий диалог с пользователем: В диалоге пользователь по имени Дана приветствует консультанта и задает вопрос о Google Analytics, на который консультант не может ответить.
****************************
Консультант:  Google Analytics — это платформа, которая собирает данные с ваших сайтов и приложений для создания отчетов, предоставляющих полезные инсайты о вашем бизнесе. С помощью
 отчетов вы можете отслеживать трафик, исследовать данные и понимать своих пользователей и их активность. Д

In [ ]:
run_dialog(system, faiss_instance)

Пользователь: Hello! How can you help me?
Консультант:  Я могу помочь вам с вопросами по Google Analytics 4, включая его функции, возможности, установку и использование. Если у вас есть конкретные вопросы или темы, которые
 вас интересуют, пожалуйста, уточните, и я постараюсь предоставить вам необходимую информацию.
Пользователь: Что такое сегменты?
****************************
Вот саммаризированный предыдущий диалог с пользователем: Пользователь приветствует консультанта и спрашивает, как тот может помочь. Консультант предлагает помощь по вопросам Google Analytics 4, включая его функции, установку и использование, и предлагает уточнить конкретные вопросы.
****************************
Консультант:  Сегменты в Google Analytics представляют собой подмножества ваших данных, которые позволяют изолировать и анализировать определенные группы пользователей, сессий или
 событий. Например, вы можете создать сегмент пользователей из конкретной страны или города, или сегмент пользователей, которы

In [ ]:
run_dialog(system, faiss_instance)


Пользователь: what is data-driven attribution?
Консультант:  Data-driven attribution — это метод распределения кредита за ключевое событие на основе данных, собранных для каждого ключевого события. Этот подход отличается от других
 моделей тем, что использует данные вашего аккаунта для расчета фактического вклада каждого взаимодействия с рекламой. Модель data-driven attribution специфична для
 каждого рекламодателя и каждого ключевого события. Она использует данные о путях пользователей, включая информацию как от конвертировавших, так и от неконвертировавших
 пользователей, чтобы понять, как наличие и время определенных маркетинговых точек касания могут повлиять на вероятность достижения ключевого события. Результирующие
 модели оценивают, насколько вероятно, что пользователь взаимодействует с ключевым событием на пути к нему в любой момент времени, учитывая воздействие конкретного
 рекламного взаимодействия. Модель также учитывает такие факторы, как время между взаимодействием с рекла

In [ ]:
run_dialog(system, faiss_instance)